In [4]:
!python3.10 -m pip install fastkml GDAL geopandas fiona pandarallel tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8/14.8 MB 19.3 MB/s eta 0:00:00a 0:00:01


In [5]:
# -----------------------------------------------------------------------------
# Initialization: Configure parallel processing
# -----------------------------------------------------------------------------

import sys
sys.path.append('..')

from baseline.utilities import *

import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.ops import unary_union
import fiona
import json
from tqdm import tqdm
from pandarallel import pandarallel

pandarallel.initialize(progress_bar=False, nb_workers=8)

fiona.drvsupport.supported_drivers['libkml'] = 'rw'
fiona.drvsupport.supported_drivers['LIBKML'] = 'rw'

kml_path = '../baseline/Building_Footprint.kml'

# Modo de ejecución: 'submission' o 'train'
MODE = 'train'

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [6]:
# -----------------------------------------------------------------------------
# Load KML File: Read building footprints and clean data
# -----------------------------------------------------------------------------

bld_footprint = gpd.read_file(kml_path, driver='libkml').replace('', np.nan)

print(bld_footprint.columns)

bld_footprint = bld_footprint.dropna(axis=1, how='all')

print(f"{bld_footprint.shape=}")
display(bld_footprint)

print(bld_footprint.crs)


Index(['Name', 'Description', 'geometry'], dtype='object')
bld_footprint.shape=(9436, 1)


,geometry
0,"MULTIPOLYGON (((-73.91903 40.8482, -73.91933 4..."
1,"MULTIPOLYGON (((-73.92195 40.84963, -73.92191 ..."
2,"MULTIPOLYGON (((-73.9205 40.85011, -73.92045 4..."
3,"MULTIPOLYGON (((-73.92056 40.8514, -73.92053 4..."
4,"MULTIPOLYGON (((-73.91234 40.85218, -73.91247 ..."
...,...
9431,"MULTIPOLYGON (((-73.95267 40.77923, -73.95254 ..."
9432,"MULTIPOLYGON (((-73.94964 40.77613, -73.94931 ..."
9433,"MULTIPOLYGON (((-73.9521 40.7688, -73.95174 40..."
9434,"MULTIPOLYGON (((-73.9523 40.75904, -73.95246 4..."


EPSG:4326


In [9]:
# -----------------------------------------------------------------------------
# Process Geometries: Convert to single polygons and compute areas
# -----------------------------------------------------------------------------

bld_footprint['geometry'] = bld_footprint['geometry'].apply(lambda x: unary_union(x))

bld_footprint = bld_footprint.to_crs(epsg=3395)

bld_footprint['area'] = bld_footprint['geometry'].area

bld_footprint = bld_footprint.to_crs(epsg=4326)

# bld_footprint.to_file('../pipeline/data/other/bf.json', driver='GeoJSON')

bbox = bld_footprint.total_bounds

In [10]:
# -----------------------------------------------------------------------------
# Load External NYC Buildings Data
# -----------------------------------------------------------------------------

bv = gpd.read_file('../baseline/BUILDING_view_-5690770882456580009.geojson')

bv = bv.to_crs(epsg=3395)

bv['area'] = bv['geometry'].area
bv = bv.to_crs(epsg=4326)

bv = bv.cx[bbox[0]:bbox[2], bbox[1]:bbox[3]].reset_index(drop=True)

bv_csv = pd.read_csv('../baseline/BUILDING_view_7607496916235021567.csv')[['DOITT ID', 'Area', 'Length']]

bv = bv.join(bv_csv.set_index('DOITT ID'), on='DOITT_ID')

bv = bv[['CONSTRUCTION_YEAR', 'FEATURE_CODE', 'GROUND_ELEVATION', 'HEIGHT_ROOF', 
         'LAST_EDITED_DATE', 'LAST_STATUS_TYPE', 'geometry', 'area', 'Area', 'Length']]

bv = bv.loc[bv['CONSTRUCTION_YEAR'] < 2021].reset_index(drop=True)

bv['height_per_squared_meter'] = bv['HEIGHT_ROOF'] / bv['area']

bv = bv[bv['geometry'].apply(lambda x: bld_footprint['geometry'].intersects(x).any())].reset_index(drop=True)

bv.to_file('../data/other/bv.json', driver='GeoJSON')


DataSourceError: ../baseline/BUILDING_view_-5690770882456580009.geojson: No such file or directory

In [ ]:
# -----------------------------------------------------------------------------
# Load Training or Submission Data and Generate Buffers
# -----------------------------------------------------------------------------

if MODE == 'train':
    ground_df = pd.read_csv('../baseline/Training_data_uhi_index.csv')
elif MODE == 'submission':
    ground_df = pd.read_csv("../baseline/Submission_template.csv")
else:
    raise ValueError("MODE debe ser 'train' o 'submission'")

ground_df.columns = ground_df.columns.str.lower()

dataset = ground_df[['longitude', 'latitude']]
dataset['geometry'] = gpd.points_from_xy(dataset['longitude'], dataset['latitude'])
geodataset = gpd.GeoDataFrame(dataset, crs='EPSG:4326')

radius_list = json.loads(open('../pipeline/data/radius_list.json', 'r').read())['radius_list']

In [ ]:
# -----------------------------------------------------------------------------
# Spatial Analysis: Compute Intersections with Buffers
# -----------------------------------------------------------------------------

for radius_meter in tqdm(radius_list, total=len(radius_list), desc='Radius Areas'):
    geodataset = geodataset.to_crs(epsg=3395)

    geodataset[f'buffer_{radius_meter}m'] = geodataset['geometry'].buffer(radius_meter)
    geodataset[f'buffer_{radius_meter}m_area'] = geodataset[f'buffer_{radius_meter}m'].area
    geodataset = geodataset.to_crs(epsg=4326)

    intersecting_squares = gpd.sjoin(
        bld_footprint, 
        gpd.GeoDataFrame(geometry=geodataset[f'buffer_{radius_meter}m'], crs=bld_footprint.crs), 
        predicate="intersects", how='inner'
    ).drop_duplicates(subset=['index_right', 'geometry'])

    squares_gb = intersecting_squares.groupby('index_right')['area']
    geodataset[f"kml_sum_areas_{radius_meter}m"] = geodataset.index.map(squares_gb.sum()).fillna(0)
